In [15]:
import requests
import numpy as np
import pandas as pd
import pandas_profiling
import json
import os
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from scipy.stats import norm
from datetime import datetime

%matplotlib inline

pd.options.display.max_columns = None

import statsmodels.api as sm # Provides cross-sectional models and methods
from statsmodels.graphics.api import abline_plot # Plots line given intercept and slope
from sklearn.metrics import mean_squared_error, r2_score # Rates how close regression line is to data
from sklearn.model_selection import train_test_split #  Splits data into training and test groups
from sklearn import linear_model, preprocessing # Ordinary least square regression and standardizes the data
import warnings # For handling error messages.

In [16]:
os.chdir(r'C:\Users\2bogu\OneDrive\Desktop\Sringboard_Materials\capstone2\data\external')

df = pd.read_csv('Green_Jobs_-_Green_New_York__GJGNY__Residential_Loan_Portfolio___Beginning_November_2010.csv')

# Initial Cleaning

In [17]:
df.columns

Index(['Reporting Period', 'CONTRACT DATE', 'SETUP DATE',
       'ORIGINAL LOAN AMOUNT', 'ORIGINAL TERM', 'ORIGINAL MATURITY DATE',
       'CURRENT MATURITY DATE', 'INTEREST RATE', 'CREDIT SCORE CURRENT HIGH',
       'CREDIT SCORE', 'CREDIT SCORE NAME 2', 'DEBT TO INCOME',
       'FIRST PAYMENT DATE', 'LAST PAYMENT DATE', 'LAST PAYMENT AMOUNT',
       'NEXT PAYMENT DUE DATE', 'PAYMENT AMOUNT', 'CURRENT BALANCE',
       'UNAPPLIED CASH', 'DELINQUENT AMOUNT', 'TOTAL INTEREST PAID',
       'TOTAL LATE CHARGE PAID', 'PAYMENTS REMAINING', 'DAYS DELINQUENT',
       'PROPERTY COUNTY', 'UTILITY', 'INSTALLATION CITY', 'INSTALLATION ZIP',
       'Contractor Name', 'SUCCESSOR NUMBER', 'ACCOUNT CODE',
       'ACCOUNT CODE DATE', 'CANCEL REASON', 'TYPE OF BANKRUPTCY',
       'Months Since Origination', 'Payments Made', 'Purpose', 'Loan Type',
       'Underwriting', 'Pledged', 'Georeference'],
      dtype='object')

In [18]:
df = df.drop(['LAST PAYMENT AMOUNT', 'LAST PAYMENT DATE', 'FIRST PAYMENT DATE', 
              'DELINQUENT AMOUNT', 'DAYS DELINQUENT', 'Reporting Period', 
              'CREDIT SCORE CURRENT HIGH', 'NEXT PAYMENT DUE DATE', 'PAYMENTS REMAINING', 
              'PROPERTY COUNTY', 'UTILITY', 'INSTALLATION CITY', 'INSTALLATION ZIP', 
              'Contractor Name', 'Georeference', 'UNAPPLIED CASH', 'TOTAL LATE CHARGE PAID', 
              'CURRENT BALANCE', 'TOTAL INTEREST PAID', 'Payments Made'], axis=1)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27385 entries, 0 to 27384
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CONTRACT DATE             27385 non-null  object 
 1   SETUP DATE                27385 non-null  object 
 2   ORIGINAL LOAN AMOUNT      27385 non-null  float64
 3   ORIGINAL TERM             27385 non-null  int64  
 4   ORIGINAL MATURITY DATE    27385 non-null  object 
 5   CURRENT MATURITY DATE     27385 non-null  object 
 6   INTEREST RATE             27385 non-null  float64
 7   CREDIT SCORE              26863 non-null  float64
 8   CREDIT SCORE NAME 2       12149 non-null  float64
 9   DEBT TO INCOME            26938 non-null  float64
 10  PAYMENT AMOUNT            27385 non-null  float64
 11  SUCCESSOR NUMBER          770 non-null    object 
 12  ACCOUNT CODE              27385 non-null  object 
 13  ACCOUNT CODE DATE         27385 non-null  object 
 14  CANCEL

In [20]:
df['CANCEL REASON'].fillna('NONE', inplace = True)

df['TYPE OF BANKRUPTCY'].fillna('NONE', inplace = True)

df['SUCCESSOR NUMBER'].fillna('NONE', inplace = True)


In [21]:
# indicates if loan has a cosigner
df['co-signed'] = np.where(pd.notnull(df['CREDIT SCORE NAME 2']), 1, 0)

# averages credit score feature to get rid of nans in credit score 2
df['avg_credit_score'] = df[['CREDIT SCORE', 'CREDIT SCORE NAME 2']].mean(axis=1)

# fills remaining na values with average of averages
df['avg_credit_score'].fillna(df['avg_credit_score'].mean(), inplace=True)

df.drop(['CREDIT SCORE','CREDIT SCORE NAME 2'], axis=1, inplace=True)

In [22]:
df['ACCOUNT CODE'].value_counts()

SurePay/Automatic Debit Payment              12061
Check/Money Order Payment (with coupon)       9612
Paid in Full                                  4903
Legal Category                                 368
Notice of Bankruptcy (written)                 186
Canceled                                       160
Soft Hold                                       66
On Hold                                         16
Management Hold                                  5
Notice of Bankruptcy (verbal)                    4
Collections Terminated (by customer)             2
Dispute of Debt                                  1
SurePay/Automatic Debit Payment (on hold)        1
Name: ACCOUNT CODE, dtype: int64

In [23]:
# FILTERING FOR FINISHED LOANS
df = df.loc[df['ACCOUNT CODE'].str.contains('Hold|Canceled|Terminated|Bankruptcy|Full', regex=True)]

In [24]:
# creates dependent var
#df['bad'] = df['CANCEL REASON'].str.contains('NONE')
df['bad'] = df['ACCOUNT CODE'].str.contains('Hold|Canceled|Terminated|Bankruptcy', regex=True)
df['bad'] = df['bad'].apply(lambda x: 1 if x==True else 0)

In [25]:
df.describe()

,ORIGINAL LOAN AMOUNT,ORIGINAL TERM,INTEREST RATE,DEBT TO INCOME,PAYMENT AMOUNT,Months Since Origination,co-signed,avg_credit_score,bad
count,5342.000000,5342.000000,5342.000000,5216.000000,5342.000000,5342.000000,5342.000000,5342.000000,5342.000000
mean,10076.636887,150.875515,3.769296,0.297223,91.328793,65.619993,0.415013,750.423501,0.082179
std,6337.052334,47.812924,0.902968,0.525935,59.209793,23.999321,0.492770,48.068659,0.274663
min,61.000000,26.000000,2.990000,0.000000,11.000000,2.000000,0.000000,527.000000,0.000000
25%,5020.250000,120.000000,3.490000,0.160000,48.000000,49.000000,0.000000,724.000000,0.000000
50%,8386.000000,180.000000,3.490000,0.280000,78.000000,63.000000,0.000000,763.500000,0.000000
75%,13431.500000,180.000000,3.490000,0.400000,123.000000,85.000000,1.000000,788.000000,0.000000
max,25000.000000,180.000000,8.490000,36.090000,455.000000,114.000000,1.000000,833.000000,1.000000


In [26]:
df['bad'].sum()

439

In [27]:
df['CANCEL REASON'].value_counts()

NONE                          5182
DEATH                           91
CUSTOMER DEFAULTED              25
SETTLEMENT                      16
CANCEL DUE TO BANKRUPTCY        13
INVOLUNTARY REPOSESSION         12
VOLUNTARY REPOSESSION            1
DEVELOPER RELEASED ACCOUNT       1
VOLUNTARY CANCEL                 1
Name: CANCEL REASON, dtype: int64

In [28]:
# cancelations due to death
91 / 439

0.2072892938496583

In [29]:
# no data collected on health
df = df[df['CANCEL REASON'] != 'DEATH']

In [30]:
df['bad'].sum()

348

In [31]:
df['SUCCESSOR NUMBER'].value_counts()

NONE    4814
S-0      310
S-1       84
S-2       28
S-3       10
S-4        2
S-5        2
s-0        1
Name: SUCCESSOR NUMBER, dtype: int64

In [32]:
dfb = df[df['bad']==1]

In [33]:
dfb['SUCCESSOR NUMBER'].value_counts()

NONE    314
S-1      17
S-0      10
S-2       4
S-3       1
S-4       1
s-0       1
Name: SUCCESSOR NUMBER, dtype: int64

In [36]:
dfb[dfb['SUCCESSOR NUMBER']!='NONE'].count()

CONTRACT DATE               34
SETUP DATE                  34
ORIGINAL LOAN AMOUNT        34
ORIGINAL TERM               34
ORIGINAL MATURITY DATE      34
CURRENT MATURITY DATE       34
INTEREST RATE               34
DEBT TO INCOME              11
PAYMENT AMOUNT              34
SUCCESSOR NUMBER            34
ACCOUNT CODE                34
ACCOUNT CODE DATE           34
CANCEL REASON               34
TYPE OF BANKRUPTCY          34
Months Since Origination    34
Purpose                     34
Loan Type                   34
Underwriting                34
Pledged                     34
co-signed                   34
avg_credit_score            34
bad                         34
dtype: int64

In [37]:
# proportion of original bad loans that are from successors
34/439

0.0774487471526196

In [40]:
# ony dealing with people who went through the application process
df = df[df['SUCCESSOR NUMBER']=='NONE']

In [41]:
#all 'customer defaulted' loans were successors
df['CANCEL REASON'].value_counts()

NONE                        4778
CANCEL DUE TO BANKRUPTCY      13
SETTLEMENT                    13
INVOLUNTARY REPOSESSION        7
CUSTOMER DEFAULTED             2
VOLUNTARY REPOSESSION          1
Name: CANCEL REASON, dtype: int64

In [ ]:
# maybe needed
df['Reporting Period'] = pd.to_datetime(df['Reporting Period'], format="%m/%d/%Y")
df['CONTRACT DATE'] = pd.to_datetime(df['CONTRACT DATE'], format="%m/%d/%Y")
df['SETUP DATE'] = pd.to_datetime(df['SETUP DATE'], format="%m/%d/%Y")
df['ORIGINAL MATURITY DATE'] = pd.to_datetime(df['ORIGINAL MATURITY DATE'], format="%m/%d/%Y")
df['CURRENT MATURITY DATE'] = pd.to_datetime(df['CURRENT MATURITY DATE'], format="%m/%d/%Y")
df['ACCOUNT CODE DATE'] = pd.to_datetime(df['ACCOUNT CODE DATE'], format="%m/%d/%Y")

In [ ]:
#df['s_y'] = df['SETUP DATE'].apply(lambda x: x.year)

In [ ]:
#df[df['s_y']==2020]

# A little EDA

In [ ]:
def diff_month(d1, d2):
    return abs((d1.year - d2.year) * 12) + abs(d1.month - d2.month)

def to_quart(d):
    qu = math.ceil(d.month/4)

In [ ]:
def time_active(df):
    if (df['ACCOUNT CODE']=='Paid in Full') | (df['bad']==1):
        return diff_month(df['ACCOUNT CODE DATE'], df['SETUP DATE'])
    else:
        return df['Months Since Origination']

In [ ]:
# to find out how long it took for loans get current account code 
# can only tell when a loan was cancelled, NOT how long a good loan has been active
# for age of good loans use Months Since Origination
df['active_m'] = df.apply(lambda x: time_active(x), axis=1)

## All

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.hist(figsize=(10,10))

## Good

In [ ]:
df_g = df[df['bad']==0]

In [ ]:
df_g.describe()

In [ ]:
sns.heatmap(df_g.corr())

In [ ]:
df_g.hist(figsize=(10,10))

## Bad

In [ ]:
df_b = df[df['bad']==1]

In [ ]:
df_b.describe()

In [ ]:
sns.heatmap(df_b.corr())

In [ ]:
df_b.hist(figsize=(10,10))

# Defining the positive Class

In [ ]:
import math
df_b['y'] = df_b['ACCOUNT CODE DATE'].apply(lambda x: x.year)

In [ ]:
df_b['y'].hist()

In [ ]:
#loans that have gone bad in 2020
df_b[df_b['y']==2020]['active_m'].hist()

In [ ]:
#oans that have gone bad before 2020
df_b[df_b['y']!=2020]['active_m'].hist()

##### The economic impact of COVID could cause people who would otherwise had paid off their loans to slip into bad standing.  This data represents only 5 months into 2020.  

___H-null___ : There is no difference in how long it took for a loan to become 'bad' loans between 2020 and other years ___Rejected(?)___

- doubling y_2020 to simulate a full year

In [ ]:
y_2020 = df_b[df_b['y']==2020]['active_m']

y_2020_ = y_2020.copy()

y_2020 = y_2020.append(y_2020_, ignore_index=True)
 
y_19 = df_b[df_b['y']!=2020]['active_m']

scipy.stats.ttest_ind(y_2020, y_19)

##### Its right on the line.  Doubling the amount of bad loans doubles both loans that have gone bad due to covid as well as loans that would have gone bad anyway.  In theory this mean the t-stat and p-value would be stronger.  They will be kept in this copy



# Defining the Negative class

## Determining threshold for loans of good standing

### DOES NOT APPLY TO THIS COPY

##### If we only include completed loans the model will not generalize well because it will be fit only on outliers.  These are 10-20 year loans and we only have 9 years of data.  The model must be able to distinguish between a typical loan and one that will fail.  It won't be able to do this if it only has extraordinary cases to train on.  When does it become safe to assume a loan will be paid off?

In [ ]:
df_b = df[df['bad']==1]

In [ ]:
import math
df_b['y'] = df_b['ACCOUNT CODE DATE'].apply(lambda x: x.year)

In [ ]:
df_b['active_m'].hist()

#q = df[df['bad']==1]['active_m'].quantile(0.95)

In [ ]:
bad_mean = np.mean(df_b.active_m)
bad_std = np.std(df_b.active_m, ddof=1)

bad_mean, bad_std

In [ ]:
df_b.active_m.quantile(q=0.99)

In [ ]:
1 - norm.cdf(95, loc=bad_mean, scale=bad_std)

In [ ]:
len(df[(df['bad'] == 0) & (df['active_m'] >= 95)])

df = df[(df['bad'] == 1) | 
        (df['ACCOUNT CODE'] == 'Paid in Full') |
        ((df['bad'] == 0) & (df['active_m'] >= 99))]

In [ ]:
len(df)

##### The number of months it takes for loans to fail is not normally distributed, so it was boostrapped with the expected mean falling within the confidence interval of the true mean.  According to that re-sampled data, there is P=0.006 chance that a loan will be canceled at or after the 99th percentile, 95 months.  We will keep loans that have been active for 95 months or more to give the model a sense of what a typical profile is, not just ones that are repaid quickly.


##### This ends up adding 1038 loans for a totla of 5503
    
   
##### Could potentially also exclude loans that were paid off a little too quickly...
    

## Defining threshold for paid off loans

##### A ten year loan thats paid off in just a couple of months is generally atypical, except in this dataset.  If there was 3rd party involvement to make this happen, and if this played a role in determining if the loan was approved or not is unknown.  There is an abundance of paid off loans compared to canceled loans and 'safe' long standing loans so defining a threshold seems to make sense.


### Sample pop = Paid off loans

In [ ]:
df_g = df[df['ACCOUNT CODE']=='Paid in Full']

In [ ]:
df_g['active_m'].hist()

In [ ]:
good_mean = np.mean(df_g.active_m)
good_std = np.std(df_g.active_m, ddof=1)

good_mean, good_std

In [ ]:
df_g.active_m.quantile(0.025)

In [ ]:
norm.cdf(0, loc=good_mean, scale=good_std)

In [ ]:
len(df[((df['ACCOUNT CODE'] == 'Paid in Full') & (df['active_m'] < 1))])

##### The p value is just barely acceptable and removes 49 loans that were paid off before a month had passed

##### MAKING NO ALTERATIONS THIS COPY

df = df[(df['ACCOUNT CODE']!= 'Paid in Full') |
        ((df['ACCOUNT CODE'] == 'Paid in Full') & (df['active_m'] >= 1))]

# A little more EDA

In [ ]:
df_b = df[(df['bad'] == 1)]
df_g = df[df['bad'] != 1]

##### Based on the histograms of each group bellow it seems it could be better to create a flag for original term length

## Redoing the initial EDA, not relevent this copy

### All

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.hist(figsize=(10,10))

### Good

In [ ]:
#df_g.describe()

In [ ]:
sns.heatmap(df_g.corr())

In [ ]:
df_g.hist(figsize=(10,10))

### Bad

In [ ]:
#df_b.describe()

In [ ]:
sns.heatmap(df_b.corr())

In [ ]:
df_b.hist(figsize=(10,10))

## Category EDA

In [ ]:
df.select_dtypes(include='object').columns

In [ ]:
objs = ['Purpose','Loan Type', 'Underwriting', 'Pledged']

for o in objs:
    plt.figure()
    sns.catplot(y=o, data=df, kind='count', hue='bad')
    plt.show()

In [ ]:
for o in objs:
    print(o)
    print(df[o].value_counts())
    print('')

In [ ]:
# one instance of a 30 debt to income ratio is removed

df = df[df['DEBT TO INCOME'] < 30]

## Scatter plots

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='avg_credit_score', y='DEBT TO INCOME', data=df, hue='bad')

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='avg_credit_score', y='ORIGINAL LOAN AMOUNT', data=df, hue='bad')

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x='avg_credit_score', y='PAYMENT AMOUNT', data=df, hue='bad')

In [ ]:
plt.figure(figsize=(10,10))
sns.violinplot(x='INTEREST RATE', y='avg_credit_score', data=df, hue='bad')

In [ ]:
plt.figure(figsize=(10,10))
sns.violinplot(x='ORIGINAL TERM', y='avg_credit_score', data=df, hue='bad')

In [ ]:
df.columns

In [ ]:
bx = ['ORIGINAL LOAN AMOUNT','ORIGINAL TERM','INTEREST RATE','DEBT TO INCOME','PAYMENT AMOUNT']

for b in bx:
    pd.DataFrame(df[b]).boxplot()
    plt.show()

# Category Binarification

In [ ]:
# CREATES BINARY COLUMN FOR PLEDGED VS UNPLEDGED LOANS
df['unpledged'] = df['Pledged'].apply(lambda x: 1 if x == 'Unpledged' else 0)

# DROPS ORIGINAL PLEDGED COLUMN
df.drop('Pledged', axis=1, inplace=True)

In [ ]:
# CREATES FLAG FOR UNDERWRITTING T1, ONLY T1 AND T2 IN SET
df['underwritten_t1'] = df['Underwriting'].apply(lambda x: 1 if x == 'Tier 1' else 0)

df.drop('Underwriting', axis=1, inplace=True)

In [ ]:
# CREATES BOOL FOR LOAN TYPE, 0 = ON BILL RECOVERY
df['loan_type_smart_energy'] = df['Loan Type'].apply(lambda x: 1 if x == 'Smart Energy' else 0)

df.drop('Loan Type', axis= 1, inplace=True)

In [ ]:
# CREATES BOOL FOR PURPOSE, DOUBLE ZERO IS OTHER
df['purpose_ee'] = df['Purpose'].apply(lambda x: 1 if x == 'Energy Efficiency (EE)' else 0)

df['purpose_sol'] = df['Purpose'].apply(lambda x: 1 if x == 'Solar (PV)' else 0)

df.drop('Purpose', axis=1, inplace=True)

In [ ]:
df.drop([
    'CONTRACT DATE', 'SETUP DATE', 'ORIGINAL MATURITY DATE',
    'CURRENT MATURITY DATE', 'ACCOUNT CODE', 'ACCOUNT CODE DATE',
    'CANCEL REASON', 'TYPE OF BANKRUPTCY','active_m','Months Since Origination' #,'y'
],
        axis=1,
        inplace=True)

In [ ]:
df['ORIGINAL TERM'].value_counts()

In [ ]:
# dtype int, but is a category. tricky tricky
df['term_180'] = df['ORIGINAL TERM'].apply(lambda x: 1 if x==180 else 0)
df['term_120'] = df['ORIGINAL TERM'].apply(lambda x: 1 if x==120 else 0)
df['term_60'] = df['ORIGINAL TERM'].apply(lambda x: 1 if x==60 else 0)

df.drop('ORIGINAL TERM',axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
(sum(df.bad))
#/len(df)

In [ ]:
#
len(df)- sum(df.bad) 


# Conclusion

##### Rows: 4813
##### Columns: 15 : 5 numeric, 10 binary categorical leave one out
##### Positive Class: 314(%6.5) : All loans that have been canceled by the original clients of GJGNY for a reason other than death or put on hold, and did not go bad in 2020
##### Negative Class: 4499: All loans paid off after one month had passed and loans in good standing that have been active for 95 months or longer

In [ ]:
df.to_csv(r'C:\Users\2bogu\OneDrive\Desktop\Sringboard_Materials\capstone2\data\interim\fl', index = False)